# Demonstrate Classification

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from CONFIG import PREPROCESSED_DATA_PATH
from knn import KNN
from gensim.models import Word2Vec
import gensim.downloader
from classifier import Classifier
from word2vec import Word2VecModel

import sklearn

In [2]:
# Set up some configs
SHOULD_RETRAIN_WORD2VEC = True
WORD2VEC_PATH = "custom_word_2_vec"

In [3]:
# Load data and shuffle
df_data = pd.read_parquet(PREPROCESSED_DATA_PATH)
df_data = df_data.sample(frac=1).reset_index(drop=True)

In [ ]:
if SHOULD_RETRAIN_WORD2VEC:
    # Configure data
    dataset_size_limt = len(df_data)#10000
    df_data = df_data[:dataset_size_limt]
    test_subset = df_data[:int(len(df_data)*0.2)]
    train_subset = df_data[int(len(df_data)*0.2):]
    
    # Fit the word2vec model
    training_tokens = np.array([x.split() for x in train_subset["title"]])
    w2v_custom = Word2VecModel()
    w2v_custom.fit(training_tokens, epochs=1024, batch_size=1024)
    
    # Save the word 2 vec model
    w2v_custom.save(WORD2VEC_PATH)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


Epoch 1/1024


# K-Fold Validation

In [ ]:
c = Classifier(WORD2VEC_PATH)

In [ ]:
df_data = df_data.sample(frac=1).reset_index(drop=True)
classes = np.array(df_data["categories"])
classes = classes[:10000]
titles = np.array(df_data["title"])
titles = titles[:10000]

In [ ]:
kf = KFold(n_splits=10)
i = 0
for train_index, test_index in kf.split(titles):
    print("FOLD ", i)
    X_train, X_test = titles[train_index], titles[test_index]
    y_train, y_test = classes[train_index], classes[test_index]
    c.fit(X_train, y_train)
    
    predictions = c.predict(X_test)
    f_score = sklearn.metrics.f1_score(y_test, predictions, average='macro')
    precision = sklearn.metrics.precision_score(y_test, predictions, average='macro')
    recall = sklearn.metrics.recall_score(y_test, predictions, average='macro')
    acc = sklearn.metrics.accuracy_score(y_test, predictions)
    print("F Score:", f_score, "Precision", precision, "Recall", recall, "Accuracy", acc)